### Import Lib

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import dateparser

### Import CSV

In [2]:
df = pd.read_csv("../data/avis/general_df.csv", index_col=0)
df = df.drop(columns=["Unnamed: 0","user"], axis=1)
df.head()

,etoiles,n_avis,localisation,date_avis,titre_avis,text_avis,date_experience,page
0,1,3.0,FR,2023-10-20T03:16:24.000Z,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
1,1,2.0,FR,2023-10-19T13:10:44.000Z,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
2,5,1.0,FR,2023-10-19T15:41:51.000Z,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
3,1,5.0,FR,2023-10-18T16:13:38.000Z,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas
4,2,4.0,FR,2023-10-17T22:22:10.000Z,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas


In [3]:
# on drop YOUNITEDCREDIT trop d'avis positifs/bots
df = df[df.page != "YounitedCredit" ]
df.head()
df.page.value_counts()

Cofidis                                 19939
OrangeBank                              14222
Floabank                                 7766
Anytime                                  6828
BoursoramaBanque                         6711
Immoprt                                  5320
Fortuneo                                 4748
Monabanq                                 4742
Oney                                     4666
Pixpay                                   4314
LaBanquePostale                          3526
MaFrenchBank                             3127
Shine                                    2429
MaBanqueBNPParibas                       1880
SocitGnrale&Vous                         1763
Hellobank!France                         1442
Kard                                     1415
SOFINCO                                  1292
LCL                                      1261
Creditmutuel                             1014
Vybe                                      938
CarrefourBanque&Assurance         

### Cleaning

In [4]:
df_cleaned = df.dropna()
df_cleaned = df_cleaned.drop(columns=["localisation","user"])
df_cleaned = df_cleaned.rename(columns={"page":"Société"})
df_cleaned.info()

KeyError: "['user'] not found in axis"

 Transformation étoiles en Int64 de 1 à 5

In [ ]:
df_cleaned["etoiles"] = pd.to_numeric(df_cleaned["etoiles"], errors="coerce")
df_cleaned = df_cleaned[df_cleaned["etoiles"].between(1, 5)] # Vérifier que les valeurs soient bien 1 à 5
df_cleaned["etoiles"] = df_cleaned["etoiles"].astype("int64")

df_cleaned.head()
df_cleaned.info()

Transformation titre et text en STR

In [5]:
df_cleaned["titre_avis"] = df_cleaned["titre_avis"].astype(str)
df_cleaned["text_avis"] = df_cleaned["text_avis"].astype(str)
df_cleaned['text_total'] = df_cleaned.titre_avis + " " + df_cleaned.text_avis
df_cleaned = df_cleaned.drop(["titre_avis","text_avis"], axis =1)
df_cleaned.head()

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_57599/3073073810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["titre_avis"] = df_cleaned["titre_avis"].astype(str)
/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_57599/3073073810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["text_avis"] = df_cleaned["text_avis"].astype(str)
/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_57599/3073073810.py:3: SettingWithCopyWarning: 
A value is trying to be

,etoiles,n_avis,localisation,date_avis,date_experience,page,text_total
0,1,3.0,FR,2023-10-20T03:16:24.000Z,Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas,Orgueil gestionnaire Ça reste une mauvaise exp...
1,1,2.0,FR,2023-10-19T13:10:44.000Z,Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...
2,5,1.0,FR,2023-10-19T15:41:51.000Z,Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas,"Chère agence , Chère agence ,je dois bous dire..."
3,1,5.0,FR,2023-10-18T16:13:38.000Z,Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas,Mépris ou incompétence ou les 2 RV pris à l'ag...
4,2,4.0,FR,2023-10-17T22:22:10.000Z,Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...


Transformation date_experience en datetime avec un dictionnaire

In [6]:
df_cleaned["date_experience"] = df_cleaned["date_experience"].str.replace("Date de l'expérience:", "").str.strip()

months_fr_to_en = {
    'janvier': 'January',
    'février': 'February',
    'mars': 'March',
    'avril': 'April',
    'mai': 'May',
    'juin': 'June',
    'juillet': 'July',
    'août': 'August',
    'septembre': 'September',
    'octobre': 'October',
    'novembre': 'November',
    'décembre': 'December'
}

def replace_month_fr_to_en(date_str):
    for fr, en in months_fr_to_en.items():
        date_str = date_str.replace(fr, en)
    return date_str

df_cleaned["date_experience"] = df_cleaned["date_experience"].apply(replace_month_fr_to_en)
df_cleaned["date_experience"] = pd.to_datetime(df_cleaned["date_experience"])

print(df_cleaned["date_experience"][0:2])

0   2023-10-19
1   2023-10-19
Name: date_experience, dtype: datetime64[ns]


In [7]:
df_cleaned.head()

,etoiles,n_avis,localisation,date_avis,date_experience,page,text_total
0,1,3.0,FR,2023-10-20T03:16:24.000Z,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire Ça reste une mauvaise exp...
1,1,2.0,FR,2023-10-19T13:10:44.000Z,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...
2,5,1.0,FR,2023-10-19T15:41:51.000Z,2023-10-19,MaBanqueBNPParibas,"Chère agence , Chère agence ,je dois bous dire..."
3,1,5.0,FR,2023-10-18T16:13:38.000Z,2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2 RV pris à l'ag...
4,2,4.0,FR,2023-10-17T22:22:10.000Z,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...


Transformation date_avis en datetime

In [9]:
# date_experience, date_avis > datetime
from datetime import datetime
df_cleaned['date_avis'] = pd.to_datetime(df_cleaned['date_avis'])

def format_date(x):
    if pd.isnull(x):
        return x  
    return x.strftime('%Y-%m-%d %H:%M:%S')

df_cleaned['date_avis'] = df_cleaned['date_avis'].apply(format_date)
df_cleaned['date_avis'] = pd.to_datetime(df_cleaned['date_avis'])
df_cleaned.head()

,etoiles,n_avis,localisation,date_avis,date_experience,page,text_total
0,1,3.0,FR,2023-10-20 03:16:24,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire Ça reste une mauvaise exp...
1,1,2.0,FR,2023-10-19 13:10:44,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...
2,5,1.0,FR,2023-10-19 15:41:51,2023-10-19,MaBanqueBNPParibas,"Chère agence , Chère agence ,je dois bous dire..."
3,1,5.0,FR,2023-10-18 16:13:38,2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2 RV pris à l'ag...
4,2,4.0,FR,2023-10-17 22:22:10,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...


In [ ]:
df_cleaned.info()

Save cleaned csv

In [10]:
df_cleaned.to_csv("../data/avis/df_cleaned.csv", encoding="utf-8", index=False)